In [58]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from IPython.display import Image  
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Image
from sklearn.preprocessing import LabelEncoder
import warnings
from sklearn import tree


In [59]:
df=pd.read_csv('Titanic.csv')

In [60]:
df.head()

,Unnamed: 0,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1st,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.337494,B5,Southampton,2,NaN,"St Louis, MO"
1,2,1st,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.550003,C22 C26,Southampton,11,NaN,"Montreal, PQ / Chesterville, ON"
2,3,1st,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.550003,C22 C26,Southampton,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,4,1st,0,"Allison, Mr. Hudson Joshua Crei",male,30.0000,1,2,113781,151.550003,C22 C26,Southampton,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,5,1st,0,"Allison, Mrs. Hudson J C (Bessi",female,25.0000,1,2,113781,151.550003,C22 C26,Southampton,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [61]:
titanic_df_trunc=df.drop(['Unnamed: 0','name','parch','ticket','fare','cabin','embarked','boat','body','home.dest'], axis=1,inplace=True)

In [62]:
titanic_df=df

In [63]:
titanic_df['sex'] = titanic_df['sex'].map({'male':'0','female':'1'})
titanic_df['survived']=titanic_df['survived'].apply(str)
titanic_df['pclass'] = titanic_df['pclass'].map({'1st':'1','2nd':'2','3rd':'3'})


In [64]:
titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    1309 non-null   object 
 1   survived  1309 non-null   object 
 2   sex       1309 non-null   object 
 3   age       1046 non-null   float64
 4   sibsp     1309 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 51.3+ KB


In [65]:
titanic_df['age'] = titanic_df['age'].fillna((titanic_df['age'].mean()))
titanic_df['age']=titanic_df['age'].apply(str)
titanic_df['sibsp']=titanic_df['sibsp'].apply(str)

In [66]:
titanic_df.dtypes

pclass      object
survived    object
sex         object
age         object
sibsp       object
dtype: object

In [67]:
X=pd.DataFrame()
X[['pclass','sex','age','sibsp']]=titanic_df[['pclass','sex','age','sibsp']]

In [68]:
X.head()

,pclass,sex,age,sibsp
0,1,1,29.0,0
1,1,0,0.916700006,1
2,1,1,2.0,1
3,1,0,30.0,1
4,1,1,25.0,1


In [69]:
y=titanic_df['survived']

In [70]:
X_train, X_test, y_train, y_test=train_test_split(X,y,train_size=0.75,test_size=0.25)

In [71]:
dec_tree=DecisionTreeClassifier()
print(dec_tree)
dec_tree.fit(X_train,y_train)



DecisionTreeClassifier()


DecisionTreeClassifier()

In [73]:
fig = plt.figure(figsize=(45,40))
_ = tree.plot_tree(dec_tree, 
                   feature_names=X_train.columns,  
                   class_names=titanic_df_trunc['survived'],
                   filled=True,
                  rounded=True,
                  fontsize=14)

TypeError: 'NoneType' object is not subscriptable

<Figure size 4500x4000 with 0 Axes>

In [ ]:
gscv_tree=GridSearchCV(dec_tree,{'criterion':['gini','entropy'],'max_depth':np.arange(3,10)},cv=10)
gscv_tree.fit(X_train,y_train)
print(gscv_tree.best_params_)

In [ ]:
gscv_tree.best_params_['criterion']

In [ ]:
new_clf=DecisionTreeClassifier(criterion=gscv_tree.best_params_['criterion'],max_depth=gscv_tree.best_params_['max_depth'])
new_model=new_clf.fit(X_train,y_train)
y_pred=new_model.predict(X_test)


In [ ]:

confusion_mat=classification_report(y_test, y_pred)
print(confusion_mat)
print("Accuracy is :",accuracy_score(y_test, y_pred)*100)
truepos = confusion_mat[1]
print(truepos)

In [ ]:
fig = plt.figure(figsize=(35,20))
_ = tree.plot_tree(new_clf, 
                   feature_names=X_train.columns,  
                   class_names=titanic_df_trunc['survived'],
                   filled=True,
                  rounded=True,
                  fontsize=15)

In [ ]:
random_clf=RandomForestClassifier(n_estimators=100,max_depth=5)
random_clf.fit(X_train,y_train)


In [ ]:
# compare results

In [ ]:

y_pred_randf=random_clf.predict(X_test)
print(classification_report(y_test,y_pred_randf))
print("New accuracy is:",accuracy_score(y_test,y_pred_randf)*100)

In [ ]:
rand_acc=accuracy_score(y_test,y_pred_randf)*100
gcv_acc=accuracy_score(y_test, y_pred)*100

In [ ]:
change=((rand_acc-gcv_acc)/rand_acc)*100

In [ ]:
print('There is approximately '+str(round(change,4))+ ' change')
